In [1]:

import pandas as pd
import math
from collections import defaultdict
from concurrent.futures import ProcessPoolExecutor, as_completed
import time
import argparse
import sympy
from functools import lru_cache
import re
import glob
import os




@lru_cache(maxsize=None)
def is_prime(num):
    return sympy.isprime(num)


def addition(number_to_generate_primes):
    """Generate primes based on the given number."""
    quotient = number_to_generate_primes
    secondmult = 0
    thirdmult = 0
    primes = []
    multiple_minus_one = 0
    twins_number = 0
    possible_twin1=0
    possible_twin2=0
    primes_plus_one=[]
    primes_minus_one=[]
    primes_plus_one_multiplier=[]
    primes_minus_one_multiplier=[]
    for index in range(2, number_to_generate_primes + 1):
        if((index & 1) == 0 or (quotient & 1) == 0):
            prime_plus_one = index * quotient + 1
            prime_minus_one = index * quotient - 1
            prime1=False
            prime2=False
            if (prime_plus_one&1==1 and is_prime(prime_plus_one)) or prime_plus_one==2:
                secondmult += 1
                prime1=True
                possible_twin1=prime_plus_one
                primes.append(prime_plus_one)
                primes_plus_one.append(prime_plus_one)
                primes_plus_one_multiplier.append(index)

            if (prime_minus_one&1==1 and is_prime(prime_minus_one)) or prime_minus_one==2:
                multiple_minus_one += 1
                prime2=True
                possible_twin2=prime_minus_one
                primes.append(prime_minus_one)
                primes_minus_one.append(prime_minus_one)
                primes_minus_one_multiplier.append(index)
            if(prime1==False or prime2==False):
                continue
            if((prime_plus_one-1) % 6!=0):
                continue
            if  (prime_plus_one-1) %6 ==0 and prime1 and prime2 and (possible_twin1 - possible_twin2 == 2):#all twin except 3,5 the composed number between the is multiple of 6n
                twins_number += 1# the 6n verification used for faster exit if not
    plus_one_csv=f"{number_to_generate_primes}+1"
    minus_one_csv=f"{number_to_generate_primes}-1"

    return secondmult, primes, multiple_minus_one, twins_number,primes_plus_one,primes_minus_one,primes_plus_one_multiplier,primes_minus_one_multiplier


def calculate_range(start, end):
    """Calculate prime-related data for a range of numbers."""
    results = []
    for number in range(start, end + 1):
        result = addition(number)
        prime_check = is_prime(number)
        results.append((number, result, prime_check))
    return results

def calculate(min_interval, end_interval):
    """Perform the main calculations and generate the data for plotting."""
    # Parallel processing
    chunk_size = 5  # Adjust the chunk size based on your system's capabilities
    ranges = [(i, min(i + chunk_size - 1, end_interval)) for i in range(min_interval, end_interval + 1, chunk_size)]
    print(ranges)
    data_to_csv={}
    data_multiplier_to_csv={}
    with ProcessPoolExecutor() as executor:
        futures = [executor.submit(calculate_range, start, end) for start, end in ranges]
        for future in as_completed(futures):
            results = future.result()
            for number, result, prime_check in results:
                if number > 0:  # Ensure number is positive
                    print(number," start")
                    plus_one_csv=f"{number}_plus_1"
                    minus_one_csv=f"{number}_minus_1"
                    data_to_csv[plus_one_csv]=result[4]
                    data_to_csv[minus_one_csv]=result[5]
                    data_multiplier_to_csv[plus_one_csv]=result[6]
                    data_multiplier_to_csv[minus_one_csv]=result[7]
                    print(number," end")




            # Create a DataFrame
    max_length = max(len(arr) for arr in data_to_csv.values())

    # Fill shorter arrays with zeros
    for key in data_to_csv:
        data_to_csv[key].extend([0] * (max_length - len(data_to_csv[key])))
    for key in data_multiplier_to_csv:
        data_multiplier_to_csv[key].extend([0] * (max_length - len(data_multiplier_to_csv[key])))



    df_primes = pd.DataFrame(data_to_csv).fillna(0)
    df = pd.DataFrame(data_multiplier_to_csv).fillna(0)

    # Write the DataFrame to a CSV file
    df_primes.to_csv(f'{min_interval} to {end_interval}primes_each_number.csv', index=False)
    df.to_csv(f'{min_interval} to {end_interval} primes_multiplier_each_number.csv', index=False)
    # Plotting

def count(start,end):
    print("counting started")


    file_path = f'{start} to {end} primes_multiplier_each_number.csv'
    df = pd.read_csv(file_path)


    # Step 2: Count occurrences
    counts = {}

    # Iterate over each column in the DataFrame
    for column in df.columns:
        values = df[column].dropna().tolist()

        for value in values:
            if value not in counts:
                counts[value] = 0
            counts[value] += 1

    # Convert the counts dictionary to a DataFrame
    counts_df = pd.DataFrame(list(counts.items()), columns=['Value', 'Count']).sort_values(by='Value')
    # Display the first few rows of the counts DataFrame
    #print(counts_df.head())
    # Step 3: Save the results
    counts_df.to_csv(f'{start} to {end}  counts .csv', index=False)

def setup(start,end):
    calculate(start, end)
    count(start,end)
    twins(start,end)
import glob
def merge_csv():
    
    # List of CSV files to merge
    csv_files = glob.glob('*counts .csv')

    # Initialize an empty DataFrame to store the merged data
    merged_df = pd.DataFrame()

    # Loop through each CSV file and merge its content
    for file in csv_files:
        df = pd.read_csv(file)
        merged_df = pd.concat([merged_df, df], ignore_index=True)

    # Group by 'Value' and sum the 'Count'
    merged_df = merged_df.groupby('Value', as_index=False).sum()

    # Save the merged DataFrame to a new CSV file
    merged_df.to_csv('merged_output.csv', index=False)

    print("CSV files merged successfully!")
def twins(start,end):
    print("twins processing begin")
    # Step 1: Read the CSV file
    input_file = f'{start} to {end} primes_multiplier_each_number.csv'
    df = pd.read_csv(input_file)

    # Step 2: Identify Column Pairs
    column_pairs = {}
    for column in df.columns:
        match = re.match(r'(\d+)_(plus|minus)_1', column)
        if match:
            base_number, suffix = match.groups()
            if base_number not in column_pairs:
                column_pairs[base_number] = {'plus': None, 'minus': None}
            column_pairs[base_number][suffix] = column

    # Step 3: Convert Columns to Sets and Calculate Intersections
    intersections = {}
    for base_number, columns in column_pairs.items():
        plus_column = columns['plus']
        minus_column = columns['minus']

        # Convert columns to sets, excluding zeros
        plus_set = set(df[plus_column][df[plus_column] != 0])
        minus_set = set(df[minus_column][df[minus_column] != 0])

        # Calculate the intersection of the sets
        intersection = plus_set.intersection(minus_set)

        # Store the intersection in a dictionary
        intersections[base_number] = intersection

    # Step 4: Create a DataFrame with Base Numbers as Headers
    # Find the maximum length of intersections to create a DataFrame with the correct shape
    max_length = max(len(intersection) for intersection in intersections.values())

    # Create a DataFrame with base numbers as headers and fill with NaN
    intersection_df = pd.DataFrame(index=range(max_length), columns=intersections.keys())

    # Fill the DataFrame with intersection values
    for base_number, intersection in intersections.items():
        intersection_df[base_number] = pd.Series(list(intersection) + [None] * (max_length - len(intersection)))

    # Step 5: Sort Values within Each Column
    for column in intersection_df.columns:
        intersection_df[column] = pd.Series(intersection_df[column].dropna().sort_values().values)

    # Step 6: Sort the Headers
    intersection_df = intersection_df.reindex(sorted(intersection_df.columns), axis=1)
    file_min_max=f"{start} to {end}"
    # Step 7: Export to CSV
    intersection_output_file = f'{file_min_max} twins_intersections_output_sorted.csv'  # Replace with your desired output file path
    intersection_df.to_csv(intersection_output_file, index=False)

    print(f"Intersections data has been exported to {intersection_output_file}")


    # Step 2: Count the Number of Values for Each Header
    value_counts = intersection_df.count()

    # Step 3: Create a DataFrame with the Counts
    counts_df = value_counts.reset_index()
    counts_df.columns = ['base_number', 'value_count']

    # Step 4: Export to CSV
    counts_output_file = f'{file_min_max} twins_value_counts_output.csv'  # Replace with your desired output file path
    counts_df.to_csv(counts_output_file, index=False)

    print(f"Value counts data has been exported to {counts_output_file}")
def main():
    #min_interval = int(input("Enter the minimum interval: "))
    #end_interval = int(input("Enter the maximum interval: "))
    start_time = time.time()
    setup(66001,67000)
    setup(66001,67000)
    setup(67001,68000)
    setup(68001,69000)
    setup(69001,70000)

    end_time = time.time()
    print(f"Elapsed time: {end_time - start_time} seconds")
    

if __name__ == "__main__":
    main()
    

[(66001, 66005), (66006, 66010), (66011, 66015), (66016, 66020), (66021, 66025), (66026, 66030), (66031, 66035), (66036, 66040), (66041, 66045), (66046, 66050), (66051, 66055), (66056, 66060), (66061, 66065), (66066, 66070), (66071, 66075), (66076, 66080), (66081, 66085), (66086, 66090), (66091, 66095), (66096, 66100), (66101, 66105), (66106, 66110), (66111, 66115), (66116, 66120), (66121, 66125), (66126, 66130), (66131, 66135), (66136, 66140), (66141, 66145), (66146, 66150), (66151, 66155), (66156, 66160), (66161, 66165), (66166, 66170), (66171, 66175), (66176, 66180), (66181, 66185), (66186, 66190), (66191, 66195), (66196, 66200), (66201, 66205), (66206, 66210), (66211, 66215), (66216, 66220), (66221, 66225), (66226, 66230), (66231, 66235), (66236, 66240), (66241, 66245), (66246, 66250), (66251, 66255), (66256, 66260), (66261, 66265), (66266, 66270), (66271, 66275), (66276, 66280), (66281, 66285), (66286, 66290), (66291, 66295), (66296, 66300), (66301, 66305), (66306, 66310), (66311,

In [2]:
import pandas as pd
from sympy import primefactors,factorint
import glob


csv_files = glob.glob('primes_factors*.csv')
def get_prime_factors(n):
    return primefactors(n,multiple=True)
    # Loop through each CSV file and merge its content



def get_complete_prime_factors(n):
    factors = factorint(n)
    complete_factors = []
    for prime, count in factors.items():
        complete_factors.extend([prime] * count)
    return complete_factors


for file in csv_files:
    df = pd.read_csv(file)
    df['Ratio'] = None
    

# Iterate through each row
    for i in range(len(df)):
        # Identify the cell with the base_number
        base_number = df.at[i, 'base_number']
        value_count= df.at[i,"value_count"]

        # Calculate the prime factors
        ratio=(value_count/base_number)*100

        # Write the prime factors in the next row of the value_count column
        df.at[i, 'Ratio'] = ratio
        df.to_csv(file ,index=False)


In [5]:
import pandas as pd
import os
import plotly.graph_objects as go

merged_df=pd.read_csv("merged_output.csv")

fig = go.Figure()

# Add a bar trace
fig.add_trace(go.Scatter(
    x=merged_df['Value'],
    y=merged_df['Count'],
    mode='markers',
    name='Count',
    marker=dict(color='blue')
))
# Update layout
fig.update_layout(
    title='Merged Counts by Value',
    xaxis_title='Value',
    yaxis_title='Count',
    bargap=0.2  # gap between bars of adjacent location coordinates
)

# Show the plot
fig.show()
fig = go.Figure()

# Add a bar trace
fig.add_trace(go.Scatter(
    x=merged_df['Count'],
    y=merged_df['Value'],
    mode='markers',
    name='Count',
    marker=dict(color='blue')
))
# Update layout
fig.update_layout(
    title='Merged Counts by Value',
    xaxis_title='Value',
    yaxis_title='Count',
    bargap=0.2  # gap between bars of adjacent location coordinates
)

# Show the plot
fig.show()

In [10]:
def count(start=20001,end=22000):
    print("counting started")
    file_path = f'{start} to {end} primes_multiplier_each_number.csv'
    df = pd.read_csv(file_path)


    # Step 2: Count occurrences
    counts = {}

    # Iterate over each column in the DataFrame
    for column in df.columns:
        values = df[column].dropna().tolist()

        for value in values:
            if value not in counts:
                counts[value] = 0
            counts[value] += 1

    # Convert the counts dictionary to a DataFrame
    counts_df = pd.DataFrame(list(counts.items()), columns=['Value', 'Count']).sort_values(by='Value')

    # Display the first few rows of the counts DataFrame
    #print(counts_df.head())
    # Step 3: Save the results
    counts_df.to_csv(f'{start} to {end}  counts .csv', index=False)
count()

counting started
